In [1]:
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_train = tf.one_hot(y_train, 10).numpy()
y_test = tf.one_hot(y_test, 10).numpy()

2022-06-02 17:31:17.418280: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-02 17:31:17.418592: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [3]:
train_dataset = dc.data.NumpyDataset(x_train, y_train)
test_dataset = dc.data.NumpyDataset(x_test, y_test)

In [4]:
features = tf.keras.Input(shape=(28,28,1))

In [5]:
conv2d_1 = layers.Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu)(features)
conv2d_2 = layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu)(conv2d_1)

In [6]:
flatten = layers.Flatten()(conv2d_2)
dense1 = layers.Dense(units=1024, activation=tf.nn.relu)(flatten)
dense2 = layers.Dense(units=10, activation=None)(dense1)

In [7]:
output = layers.Activation(tf.math.softmax)(dense2)

In [8]:
keras_model = tf.keras.Model(inputs=features, outputs=[output, dense2])

In [9]:
model = dc.models.KerasModel(keras_model, loss=dc.models.losses.SoftmaxCrossEntropy(),
                            output_types=['prediction', 'loss'],
                            model_dir='mnist')

In [10]:
model.fit(train_dataset, nb_epoch=1)

0.10718879699707032

In [11]:
metric = dc.metrics.Metric(dc.metrics.accuracy_score)

In [12]:
train_scores = model.evaluate(train_dataset, [metric])
test_scores = model.evaluate(test_dataset, [metric])

In [13]:
test_scores

{'accuracy_score': 0.99367}

In [14]:
train_scores

{'accuracy_score': 0.9952483333333333}